In [ ]:
import yaml
import datetime as dt

import pandas as pd
import numpy as np

from bokeh.layouts import column
from bokeh.models import (ColumnDataSource, Slider, LabelSet, CategoricalColorMapper, DataTable, HoverTool, IntEditor,
                          NumberEditor, NumberFormatter, SelectEditor,
                          StringEditor, StringFormatter, TableColumn, LinearColorMapper)
from bokeh.plotting import figure
from bokeh.themes import Theme
from bokeh.palettes import Spectral4, viridis, Viridis256
from bokeh.io import show, output_notebook
from bokeh.transform import jitter, factor_cmap, linear_cmap

from squarify import normalize_sizes, squarify
import colorcet as cc


output_notebook()

In [ ]:
## IMPORT IO TRACES AND CREATE TWO DATAFRAMES

io_traces = None
with open("timestamped_io_operations.yml", "r", encoding="utf-8") as traces:
    io_traces = yaml.load(traces, Loader=yaml.SafeLoader)
    
ts_traces = pd.DataFrame(io_traces)
ts_traces

In [ ]:
dummy = {
    'ts': [1, 1, 10, 10, 1, 1, 10, 10], 
    'storage_service_name': ["storage1", "storage1", "storage1", "storage1", "storage2", "storage2", "storage2", "storage2"], 
    'disk': ["disk0", "disk1", "disk0", "disk1", "hdd1", "hdd2", "hdd1", "hdd2"],
    'capacity': [100, 150, 100, 150, 200, 120, 200, 120], 
    'free_space': [100, 150, 90, 110, 200, 120, 80, 20], 
    'allocations': [0, 0, 1, 1, 0, 0, 1, 1]
}
df_dummy = pd.DataFrame(dummy).sort_values(by="ts")
df_dummy

In [ ]:
unique_ts = pd.Series(df_dummy["ts"].unique()).sort_values()
unique_ts

In [ ]:
import matplotlib


df_dummy_1 = df_dummy[df_dummy["ts"] == unique_ts[0]]
df_dummy_1 = df_dummy_1[["storage_service_name", "disk", "capacity", "free_space"]]
df_dummy_1["percent_free"] = df_dummy_1["free_space"].mul(100) 
df_dummy_1["percent_free"] = df_dummy_1["percent_free"] / df_dummy_1["capacity"]
print(df_dummy_1)
print("\n\n")

df_dummy_1_by_server = df_dummy_1[["storage_service_name", "disk", "capacity"]].groupby("storage_service_name").sum("capacity").sort_values(by="capacity")
print(df_dummy_1_by_server)
print("\n\n")

server_names = ("storage1", "storage2")

def treemap(df, col, x, y, dx, dy):
    sub_df = df.copy() # nlargest(N, col)
    normed = normalize_sizes(sub_df[col], dx, dy)
    blocks = squarify(normed, x, y, dx, dy)
    blocks_df = pd.DataFrame.from_dict(blocks).set_index(sub_df.index)
    return sub_df.join(blocks_df, how="left").reset_index()

x, y, w, h = 0, 0, 800, 450

blocks_by_server = treemap(df_dummy_1_by_server, "capacity", x, y, w, h)

print(blocks_by_server)
print("\n\n")

dfs = []
for index, (storage_server, capacity, x, y, dx, dy) in blocks_by_server.iterrows():
    df = df_dummy_1[df_dummy_1.storage_service_name==storage_server]
    dfs.append(treemap(df, "capacity", x, y, dx, dy))
blocks = pd.concat(dfs)

p = figure(width=w, height=h, tooltips="@storage_service_name,@disk", toolbar_location=None,
           x_axis_location=None, y_axis_location=None)
p.x_range.range_padding = p.y_range.range_padding = 0
p.grid.grid_line_color = None

p.block('x', 'y', 'dx', 'dy', source=blocks, line_width=3, line_color="black",
        fill_alpha=0.8, fill_color=factor_cmap("storage_service_name", "MediumContrast4", server_names))

colors = ["#3aeb34", "#ffba26", "#ff2a26"][::-1]
mapper = LinearColorMapper(palette=colors, low=0, high=100)

internal_blocks = blocks.copy()
print(internal_blocks)
internal_blocks["x"] = internal_blocks["x"] + 20
internal_blocks["y"] = internal_blocks["y"] + 20
internal_blocks["dx"] = internal_blocks["dx"] - 40
internal_blocks["dy"] = internal_blocks["dy"] - 40
p.block('x', 'y', 'dx', 'dy', source=internal_blocks, line_width=2, line_color="black",
        fill_alpha=0.8, fill_color={'field': 'percent_free', 'transform': mapper}) #fill_color=linear_cmap('percent_free', mapper, 0, 100))

p.text('x', 'y', x_offset=2, text="storage_service_name", source=blocks_by_server,
       text_font_size="18pt", text_color="white")

blocks["ytop"] = blocks.y + blocks.dy
p.text('x', 'ytop', x_offset=2, y_offset=2, text="disk", source=blocks,
       text_font_size="12pt", text_baseline="top")
       #text_color=factor_cmap("Region", ("black", "white", "black", "white"), regions))

show(p)